In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

from sqlalchemy import create_engine

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\JK\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [3]:
# Visit website
url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8871119/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, "html.parser")

In [5]:
#Scraping the webpage to pull the url for individual plants
wine_header = []

wine_sample_table = soup.find('div', class_="table-wrap anchored whole_rhythm")
wine_sample_data = wine_sample_table.find_all('tr')[0]

for i in range(0, len(wine_sample_data)):
    wine_dict = {}
    wine_dict = wine_sample_data.find_all('th')[i].text
    wine_header.append(wine_dict)
print(wine_header)

['Label', 'Product Name', 'Cultivar', 'Region', 'Vintage', 'Alcohol', 'Sugar Content']


In [6]:
#Scraping the webpage to pull the url for individual plants

wine_list = []
wine_sample_table = soup.find('div', class_="table-wrap anchored whole_rhythm")

for i in range (0,11):
    wine_data = wine_sample_table.find_all('tr')[i+2]

    wine_dict = {}
    for j in range(0,7):
        wine_dict[wine_header[j]] = wine_data.find_all('td')[j].text
    wine_list.append(wine_dict)
        
print(wine_list)

[{'Label': 'W1', 'Product Name': 'Famille Hugel, Gewurztraminer Classic', 'Cultivar': 'Gewurztraminer', 'Region': 'France', 'Vintage': '2015', 'Alcohol': '14', 'Sugar Content': '8.03'}, {'Label': 'W2', 'Product Name': 'Mouton Cadet White', 'Cultivar': 'Sauvignon Blanc, Sémillon, Muscadelle', 'Region': 'France', 'Vintage': '2017', 'Alcohol': '12', 'Sugar Content': '6.3'}, {'Label': 'W3', 'Product Name': 'Albert Bichot Chablis Primier Cru Les Vaucopins', 'Cultivar': 'Chablis', 'Region': 'France', 'Vintage': '2017', 'Alcohol': '13', 'Sugar Content': '6.63'}, {'Label': 'W4', 'Product Name': 'Kressmann Grand Reserve Bordeaux', 'Cultivar': 'Sauvignon Blanc, Muscadelle, Sémillon', 'Region': 'France', 'Vintage': '2017', 'Alcohol': '12.5', 'Sugar Content': '6.37'}, {'Label': 'W5', 'Product Name': 'Majuang Mosel', 'Cultivar': 'Riesling', 'Region': 'Germany', 'Vintage': '2017', 'Alcohol': '8', 'Sugar Content': '7.83'}, {'Label': 'W6', 'Product Name': 'Schloss Vollrads, Edition/Riseling', 'Cultiva

In [7]:
browser.quit()

In [8]:
df = pd.DataFrame(wine_list)
df.head()

,Label,Product Name,Cultivar,Region,Vintage,Alcohol,Sugar Content
0,W1,"Famille Hugel, Gewurztraminer Classic",Gewurztraminer,France,2015,14,8.03
1,W2,Mouton Cadet White,"Sauvignon Blanc, Sémillon, Muscadelle",France,2017,12,6.3
2,W3,Albert Bichot Chablis Primier Cru Les Vaucopins,Chablis,France,2017,13,6.63
3,W4,Kressmann Grand Reserve Bordeaux,"Sauvignon Blanc, Muscadelle, Sémillon",France,2017,12.5,6.37
4,W5,Majuang Mosel,Riesling,Germany,2017,8,7.83


In [9]:
df = df.drop(columns=["Label"],axis=1)
df = df.rename(columns={"Product Name":"product_name", "Cultivar":"cultivar","Region":"region","Vintage":"vintage","Alcohol":"alcohol","Sugar Content":"residual_sugar"})
df.head(11)

,product_name,cultivar,region,vintage,alcohol,residual_sugar
0,"Famille Hugel, Gewurztraminer Classic",Gewurztraminer,France,2015,14,8.03
1,Mouton Cadet White,"Sauvignon Blanc, Sémillon, Muscadelle",France,2017,12,6.3
2,Albert Bichot Chablis Primier Cru Les Vaucopins,Chablis,France,2017,13,6.63
3,Kressmann Grand Reserve Bordeaux,"Sauvignon Blanc, Muscadelle, Sémillon",France,2017,12.5,6.37
4,Majuang Mosel,Riesling,Germany,2017,8,7.83
5,"Schloss Vollrads, Edition/Riseling",Riesling,Germany,2016,12.5,7.6
6,Marchesi Antinori Villa Antinori Bianco,"Trebbiano, Malvasia, Chardonnay",Italy,2017,12,6.5
7,LE RIME Banfi,"Chardonnay, Pinot Grigio",Italy,2018,12,6.83
8,Cloudy Bay Sauvignon Blanc,Sauvignon Blanc,New Zealand,2018,13,7
9,Torres Vina Sol,"Parellada, Garnacha Blanca",Spain,2017,11.5,6.2


In [10]:
df['product_name'] = df['product_name'].str.replace('/', '')
df

,product_name,cultivar,region,vintage,alcohol,residual_sugar
0,"Famille Hugel, Gewurztraminer Classic",Gewurztraminer,France,2015,14,8.03
1,Mouton Cadet White,"Sauvignon Blanc, Sémillon, Muscadelle",France,2017,12,6.3
2,Albert Bichot Chablis Primier Cru Les Vaucopins,Chablis,France,2017,13,6.63
3,Kressmann Grand Reserve Bordeaux,"Sauvignon Blanc, Muscadelle, Sémillon",France,2017,12.5,6.37
4,Majuang Mosel,Riesling,Germany,2017,8,7.83
5,"Schloss Vollrads, EditionRiseling",Riesling,Germany,2016,12.5,7.6
6,Marchesi Antinori Villa Antinori Bianco,"Trebbiano, Malvasia, Chardonnay",Italy,2017,12,6.5
7,LE RIME Banfi,"Chardonnay, Pinot Grigio",Italy,2018,12,6.83
8,Cloudy Bay Sauvignon Blanc,Sauvignon Blanc,New Zealand,2018,13,7
9,Torres Vina Sol,"Parellada, Garnacha Blanca",Spain,2017,11.5,6.2


In [11]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [12]:
Base = declarative_base()

In [13]:
class wine(Base):
    __tablename__ = 'wine_table'
    id = Column(Integer, primary_key=True)
    product_name = Column(String(10))
    cultivar = Column(String(100))
    region = Column(String(30))
    vintage = Column(String(30))
    alcohol = Column(Float)
    residual_sugar = Column(Float)


In [14]:
#Create db connection
engine = create_engine("sqlite:///wine_data.sqlite")
conn = engine.connect()

In [15]:
#creates tables from python classes
Base.metadata.create_all(engine)

In [16]:
#create connection to edit db
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [17]:
#for loop this with scraped data to add to db
for row in df.itertuples():
    wine_data_add = wine(product_name=row[1], cultivar=row[2], region=row[3], vintage=row[4], alcohol=row[5], residual_sugar=row[6])
    session.add(wine_data_add)

In [18]:
#push additions to db
session.commit()

In [19]:
#close session
session.close()

In [20]:
# wine.__table__.drop(engine)

In [21]:
session.query(wine.product_name).all()

[('Famille Hugel, Gewurztraminer Classic',),
 ('Mouton Cadet White',),
 ('Albert Bichot Chablis Primier Cru Les Vaucopins',),
 ('Kressmann Grand Reserve Bordeaux',),
 ('Majuang Mosel',),
 ('Schloss Vollrads, EditionRiseling',),
 ('Marchesi Antinori Villa Antinori Bianco',),
 ('LE RIME Banfi',),
 ('Cloudy Bay Sauvignon Blanc',),
 ('Torres Vina Sol',),
 ('Kendall Jackson, Grand Reserve Chardonnay 3',)]